In this notebook, we analyze the results of NGCF trained using the Elliot framework.

In [ ]:
import pandas as pd

pd.options.display.max_rows = 100
import os
from data_generation import generate_data
from itertools import chain
import modelling_mf

In [ ]:
data = "fairbook"
# user-item interactions
fairbook_ratings = pd.read_csv("data/" + data + "_events.csv")

In [ ]:
user_col = "user"  # the name of the column that includes the users
item_col = "item"  # the name of the column that includes the items
predict_col = "rating"  # the name of the column that includes the interaction

In [ ]:
data_strategies = [
    "uniformly_random",
    "popularity_good",
    "popularity_bad",
    "popularity_good_for_bp_ur",
    "popularity_bad_for_bp_ur",
]

In [ ]:
location = "Elliotresults/"

In [ ]:
start = 0
keep_interval = 10
skip_interval = 40

# Uniformly random

In [ ]:
added_location = "UniRand-"
j = 0

In [ ]:
ratings = generate_data(
    strategy=data_strategies[j], copying_dataset=fairbook_ratings, user_perc=0.2
)
all_items = set(ratings.item.unique())
# sample = xf.SampleFrac(0.2, rng_spec=0)
# sets = [i for i in enumerate(xf.partition_users(ratings,5, sample,rng_spec=0))]

In [ ]:
ur_metrics = []
for i in range(5):
    current_location = location + added_location + str(i + 1) + "/"
    print(current_location)
    result_file = current_location + os.listdir(current_location)[0]
    print(result_file)
    # sett = sets[i]

    train_df = pd.read_csv(
        "all_folds/" + data_strategies[j] + "_fold_" + str(i + 1) + "_train.csv"
    )
    test_df = pd.read_csv(
        "all_folds/" + data_strategies[j] + "_fold_" + str(i + 1) + "_test.csv"
    )

    # train_df = sett[1][0]
    # test_df = sett[1][1]
    test_users = test_df.user.unique()
    result_df = pd.read_csv(result_file, sep="\t", header=None)
    result_df.columns = ["user", "item", "rating"]
    test_result = result_df[result_df.user.isin(test_users)].reset_index(drop=True)

    end = len(test_result)
    # Create an iterable of indices to keep
    indices_to_keep = list(
        chain.from_iterable(
            range(t, t + keep_interval)
            for t in range(start, end, keep_interval + skip_interval)
        )
    )
    filtered_test_result = test_result.iloc[indices_to_keep].reset_index(
        drop=True
    )  # keep top 10
    # filtered_test_result = test_result.copy()
    recs_grouped = filtered_test_result.groupby([user_col])[item_col].apply(list)
    pop_bias = modelling_mf.calculate_pop_bias_per_item(
        all_items, item_col, user_col, predict_col, train_df, recs=filtered_test_result
    )
    GAP_vs_GAP = modelling_mf.calculate_ave_pop_per_user(
        test_users, item_col, user_col, pop_bias, train_df, recs_grouped
    )
    pop_corr = modelling_mf.calculate_pop_correlation(pop_bias)
    precision, recall, ndcg = modelling_mf.calculate_topn_metrics(
        filtered_test_result, test_df
    )
    AggDiv = modelling_mf.evaluate_item_coverage(pop_bias["recommendation"].values)
    ARP, ave_PL, ACLT = modelling_mf.calculate_all_pb_metrics(
        pop_bias,
        test_users,
        item_col,
        user_col,
        train_df,
        recs_grouped,
        filtered_test_result,
    )
    metrics_dict_ur = {
        "pop_corr": pop_corr,
        "RMSE": 0,
        "NDCG": ndcg,
        "ARP": ARP,
        "ave_PL": ave_PL,
        "ACLT": ACLT,
        "AggDiv": AggDiv,
    }
    ur_metrics.append(metrics_dict_ur)
    pop_biases = [pop_bias]
    modelling_mf.plot_results(
        pop_biases.copy(),
        GAP_vs_GAP.copy(),
        "CornacNGCF",
        0,
        precision,
        recall,
        ndcg,
        0,
        0,
        cv=False,
        n=10,
        args="fold" + str(i + 1),
        data_strategy=data_strategies[j],
        save_plot=True,
    )

# Popularity Good

In [ ]:
added_location = "GoodPop-"
j = 1

In [ ]:
# ratings = generate_data(strategy = data_strategies[j],
#                             copying_dataset = fairbook_ratings,
#                             user_perc = 0.2)
# all_items=set(ratings.item.unique())
# sample = xf.SampleFrac(0.2, rng_spec=0)
# sets = [i for i in enumerate(xf.partition_users(ratings,5, sample,rng_spec=0))]

In [ ]:
pg_metrics = []
for i in range(5):
    current_location = location + added_location + str(i + 1) + "/"
    print(current_location)
    if os.listdir(current_location)[0] == ".ipynb_checkpoints":
        result_file = current_location + os.listdir(current_location)[1]
    else:
        result_file = current_location + os.listdir(current_location)[0]
    print(result_file)
    # sett = sets[i]

    train_df = pd.read_csv(
        "all_folds/" + data_strategies[j] + "_fold_" + str(i + 1) + "_train.csv"
    )
    test_df = pd.read_csv(
        "all_folds/" + data_strategies[j] + "_fold_" + str(i + 1) + "_test.csv"
    )

    # train_df = sett[1][0]
    # test_df = sett[1][1]
    test_users = test_df.user.unique()
    result_df = pd.read_csv(result_file, sep="\t", header=None)
    result_df.columns = ["user", "item", "rating"]
    test_result = result_df[result_df.user.isin(test_users)].reset_index(drop=True)

    end = len(test_result)
    # Create an iterable of indices to keep
    indices_to_keep = list(
        chain.from_iterable(
            range(t, t + keep_interval)
            for t in range(start, end, keep_interval + skip_interval)
        )
    )
    filtered_test_result = test_result.iloc[indices_to_keep].reset_index(
        drop=True
    )  # keep top 10
    # filtered_test_result = test_result.copy()
    recs_grouped = filtered_test_result.groupby([user_col])[item_col].apply(list)
    pop_bias = modelling_mf.calculate_pop_bias_per_item(
        all_items, item_col, user_col, predict_col, train_df, recs=filtered_test_result
    )
    GAP_vs_GAP = modelling_mf.calculate_ave_pop_per_user(
        test_users, item_col, user_col, pop_bias, train_df, recs_grouped
    )
    pop_corr = modelling_mf.calculate_pop_correlation(pop_bias)
    precision, recall, ndcg = modelling_mf.calculate_topn_metrics(
        filtered_test_result, test_df
    )
    AggDiv = modelling_mf.evaluate_item_coverage(pop_bias["recommendation"].values)
    ARP, ave_PL, ACLT = modelling_mf.calculate_all_pb_metrics(
        pop_bias,
        test_users,
        item_col,
        user_col,
        train_df,
        recs_grouped,
        filtered_test_result,
    )
    metrics_dict_pg = {
        "pop_corr": pop_corr,
        "RMSE": 0,
        "NDCG": ndcg,
        "ARP": ARP,
        "ave_PL": ave_PL,
        "ACLT": ACLT,
        "AggDiv": AggDiv,
    }
    pg_metrics.append(metrics_dict_pg)
    pop_biases = [pop_bias]
    modelling_mf.plot_results(
        pop_biases.copy(),
        GAP_vs_GAP.copy(),
        "CornacNGCF",
        0,
        precision,
        recall,
        ndcg,
        0,
        0,
        cv=False,
        n=10,
        args="fold" + str(i + 1),
        data_strategy=data_strategies[j],
        save_plot=True,
    )

# Popularity Bad

In [ ]:
added_location = "badPop-"
j = 2

In [ ]:
pb_metrics = []
for i in range(5):
    current_location = location + added_location + str(i + 1) + "/"
    print(current_location)
    if os.listdir(current_location)[0] == ".DS_Store":
        result_file = current_location + os.listdir(current_location)[1]
    else:
        result_file = current_location + os.listdir(current_location)[0]
    print(result_file)
    # sett = sets[i]

    train_df = pd.read_csv(
        "all_folds/" + data_strategies[j] + "_fold_" + str(i + 1) + "_train.csv"
    )
    test_df = pd.read_csv(
        "all_folds/" + data_strategies[j] + "_fold_" + str(i + 1) + "_test.csv"
    )

    # train_df = sett[1][0]
    # test_df = sett[1][1]
    test_users = test_df.user.unique()
    result_df = pd.read_csv(result_file, sep="\t", header=None)
    result_df.columns = ["user", "item", "rating"]
    test_result = result_df[result_df.user.isin(test_users)].reset_index(drop=True)

    end = len(test_result)
    # Create an iterable of indices to keep
    indices_to_keep = list(
        chain.from_iterable(
            range(t, t + keep_interval)
            for t in range(start, end, keep_interval + skip_interval)
        )
    )
    filtered_test_result = test_result.iloc[indices_to_keep].reset_index(
        drop=True
    )  # keep top 10
    # filtered_test_result = test_result.copy()
    recs_grouped = filtered_test_result.groupby([user_col])[item_col].apply(list)
    pop_bias = modelling_mf.calculate_pop_bias_per_item(
        all_items, item_col, user_col, predict_col, train_df, recs=filtered_test_result
    )
    GAP_vs_GAP = modelling_mf.calculate_ave_pop_per_user(
        test_users, item_col, user_col, pop_bias, train_df, recs_grouped
    )
    pop_corr = modelling_mf.calculate_pop_correlation(pop_bias)
    precision, recall, ndcg = modelling_mf.calculate_topn_metrics(
        filtered_test_result, test_df
    )
    AggDiv = modelling_mf.evaluate_item_coverage(pop_bias["recommendation"].values)
    ARP, ave_PL, ACLT = modelling_mf.calculate_all_pb_metrics(
        pop_bias,
        test_users,
        item_col,
        user_col,
        train_df,
        recs_grouped,
        filtered_test_result,
    )
    metrics_dict_pb = {
        "pop_corr": pop_corr,
        "RMSE": 0,
        "NDCG": ndcg,
        "ARP": ARP,
        "ave_PL": ave_PL,
        "ACLT": ACLT,
        "AggDiv": AggDiv,
    }
    pb_metrics.append(metrics_dict_pb)
    pop_biases = [pop_bias]
    modelling_mf.plot_results(
        pop_biases.copy(),
        GAP_vs_GAP.copy(),
        "CornacNGCF",
        0,
        precision,
        recall,
        ndcg,
        0,
        0,
        cv=False,
        n=10,
        args="fold" + str(i + 1),
        data_strategy=data_strategies[j],
        save_plot=True,
    )

# Popularity Good for Bp Ur

In [ ]:
added_location = "bp_ur-GoodPop-"
j = 3

In [ ]:
pgbp_metrics = []
for i in range(5):
    current_location = location + added_location + str(i + 1) + "/"
    print(current_location)
    result_file = current_location + os.listdir(current_location)[0]
    print(result_file)
    # sett = sets[i]

    train_df = pd.read_csv(
        "all_folds/" + data_strategies[j] + "_fold_" + str(i + 1) + "_train.csv"
    )
    test_df = pd.read_csv(
        "all_folds/" + data_strategies[j] + "_fold_" + str(i + 1) + "_test.csv"
    )

    # train_df = sett[1][0]
    # test_df = sett[1][1]
    test_users = test_df.user.unique()
    result_df = pd.read_csv(result_file, sep="\t", header=None)
    result_df.columns = ["user", "item", "rating"]
    test_result = result_df[result_df.user.isin(test_users)].reset_index(drop=True)

    end = len(test_result)
    # Create an iterable of indices to keep
    indices_to_keep = list(
        chain.from_iterable(
            range(t, t + keep_interval)
            for t in range(start, end, keep_interval + skip_interval)
        )
    )
    filtered_test_result = test_result.iloc[indices_to_keep].reset_index(
        drop=True
    )  # keep top 10
    # filtered_test_result = test_result.copy()
    recs_grouped = filtered_test_result.groupby([user_col])[item_col].apply(list)
    pop_bias = modelling_mf.calculate_pop_bias_per_item(
        all_items, item_col, user_col, predict_col, train_df, recs=filtered_test_result
    )
    GAP_vs_GAP = modelling_mf.calculate_ave_pop_per_user(
        test_users, item_col, user_col, pop_bias, train_df, recs_grouped
    )
    pop_corr = modelling_mf.calculate_pop_correlation(pop_bias)
    precision, recall, ndcg = modelling_mf.calculate_topn_metrics(
        filtered_test_result, test_df
    )
    AggDiv = modelling_mf.evaluate_item_coverage(pop_bias["recommendation"].values)
    ARP, ave_PL, ACLT = modelling_mf.calculate_all_pb_metrics(
        pop_bias,
        test_users,
        item_col,
        user_col,
        train_df,
        recs_grouped,
        filtered_test_result,
    )
    metrics_dict_pgbp = {
        "pop_corr": pop_corr,
        "RMSE": 0,
        "NDCG": ndcg,
        "ARP": ARP,
        "ave_PL": ave_PL,
        "ACLT": ACLT,
        "AggDiv": AggDiv,
    }
    pgbp_metrics.append(metrics_dict_pgbp)
    pop_biases = [pop_bias]
    modelling_mf.plot_results(
        pop_biases.copy(),
        GAP_vs_GAP.copy(),
        "CornacNGCF",
        0,
        precision,
        recall,
        ndcg,
        0,
        0,
        cv=False,
        n=10,
        args="fold" + str(i + 1),
        data_strategy=data_strategies[j],
        save_plot=True,
    )

# Popularity Bad for Bp Ur

In [ ]:
added_location = "bp_ur-BadPop-"
j = 4

In [ ]:
pbbp_metrics = []
for i in range(5):
    current_location = location + added_location + str(i + 1) + "/"
    print(current_location)
    if os.listdir(current_location)[0] == ".DS_Store":
        result_file = current_location + os.listdir(current_location)[1]
    else:
        result_file = current_location + os.listdir(current_location)[0]
    print(result_file)
    # sett = sets[i]

    train_df = pd.read_csv(
        "all_folds/" + data_strategies[j] + "_fold_" + str(i + 1) + "_train.csv"
    )
    test_df = pd.read_csv(
        "all_folds/" + data_strategies[j] + "_fold_" + str(i + 1) + "_test.csv"
    )

    # train_df = sett[1][0]
    # test_df = sett[1][1]
    test_users = test_df.user.unique()
    result_df = pd.read_csv(result_file, sep="\t", header=None)
    result_df.columns = ["user", "item", "rating"]
    test_result = result_df[result_df.user.isin(test_users)].reset_index(drop=True)

    end = len(test_result)
    # Create an iterable of indices to keep
    indices_to_keep = list(
        chain.from_iterable(
            range(t, t + keep_interval)
            for t in range(start, end, keep_interval + skip_interval)
        )
    )
    filtered_test_result = test_result.iloc[indices_to_keep].reset_index(
        drop=True
    )  # keep top 10
    # filtered_test_result = test_result.copy()
    recs_grouped = filtered_test_result.groupby([user_col])[item_col].apply(list)
    pop_bias = modelling_mf.calculate_pop_bias_per_item(
        all_items, item_col, user_col, predict_col, train_df, recs=filtered_test_result
    )
    GAP_vs_GAP = modelling_mf.calculate_ave_pop_per_user(
        test_users, item_col, user_col, pop_bias, train_df, recs_grouped
    )
    pop_corr = modelling_mf.calculate_pop_correlation(pop_bias)
    precision, recall, ndcg = modelling_mf.calculate_topn_metrics(
        filtered_test_result, test_df
    )
    AggDiv = modelling_mf.evaluate_item_coverage(pop_bias["recommendation"].values)
    ARP, ave_PL, ACLT = modelling_mf.calculate_all_pb_metrics(
        pop_bias,
        test_users,
        item_col,
        user_col,
        train_df,
        recs_grouped,
        filtered_test_result,
    )
    metrics_dictpbbp = {
        "pop_corr": pop_corr,
        "RMSE": 0,
        "NDCG": ndcg,
        "ARP": ARP,
        "ave_PL": ave_PL,
        "ACLT": ACLT,
        "AggDiv": AggDiv,
    }

    pbbp_metrics.append(metrics_dictpbbp)
    pop_biases = [pop_bias]
    modelling_mf.plot_results(
        pop_biases.copy(),
        GAP_vs_GAP.copy(),
        "CornacNGCF",
        0,
        precision,
        recall,
        ndcg,
        0,
        0,
        cv=False,
        n=10,
        args="fold" + str(i + 1),
        data_strategy=data_strategies[j],
        save_plot=True,
    )

# Combine results

In [ ]:
metrics_list = [ur_metrics, pg_metrics, pb_metrics, pgbp_metrics, pbbp_metrics]

In [ ]:
relevant_values = ["pop_corr", "ARP", "ave_PL", "AggDiv", "NDCG"]

In [ ]:
strategy_list = []
for strategy in metrics_list:
    curr_dict = {}
    for value in relevant_values:
        ave_value = sum(d[value] for d in strategy) / len(strategy)
        curr_dict[value] = ave_value
    strategy_list.append(curr_dict)

In [ ]:
ds = ["Scenario 1", "Scenario 2", "Scenario 3", "Scenario 4", "Scenario 5"]

In [ ]:
result = pd.DataFrame(strategy_list, index=ds)

In [ ]:
print(result.round(3).to_latex())

In [ ]:
import pickle

with open("metrics_combined/all_ngcf.pkl", "wb") as f:
    pickle.dump(result.round(3), f)